# Importar Librerias

In [ ]:
import datetime
import pickle

import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [ ]:
!pip install openpyxl

# Cargar Datasets

In [ ]:
# Data Autorizaciones

df = pd.read_parquet(
    '../data/raw/DataAutorizacionesNov2021Mar2022.parquet', 
)

In [ ]:
# Data Distribuciones Autorizaciones

df2 = pd.read_parquet(
    '../data/raw/DataDistribucionesAutorizacionesNov2021Mar2022.parquet', 
)

In [ ]:
# CUPS frecuentes en emergencias
cups_df = pd.read_excel(
    '../data/raw/CUPS frecuencias y otros (002).xlsx',engine='openpyxl'
)

In [ ]:
# CUPS cantidades esperadas
valores_referencia_cups_df = pd.read_excel(
    '../data/raw/CUPS frecuencias y otros (002).xlsx', 
    sheet_name=1,engine='openpyxl'
)

In [ ]:
# Indicador de dias de habitación

# Read File
salas_df = pd.read_excel(
    '../data/raw/salas.xlsx', engine='openpyxl'
)

salas_list = salas_df['DESCRIPCION'].tolist()

/anaconda/envs/azureml_py38/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


### Unir datasets

In [ ]:
# Extraer autorizaciones en los dos datasets
autorizaciones_1 = set(df.AUTORIZACION.unique())
autoriaciones_2 = set(df2.AUTORIZACION.unique())
autorizaciones = autorizaciones_1.intersection(autoriaciones_2)

tmp_df_columns = [
    'AUTORIZACION', 
    'CLIENTEAFILIADOBENEFICIARIO',
    'PRESTACION', 
    'PRESTADOR', 
    'FECHASERVICIO', 
    'MONTORECLAMADO'
]

tmp_df2_columns = [
    'AUTORIZACION', 
    'CLIENTEAFILIADOBENEFICIARIO', 
    'PRESTACION', 
    'PRESTADOR',
    'FECHASERVICIO', 
    'MONTOASEGURADORA', 
    'MONTOEXCEPCION', 
    'MONTOCONCESION', 
    'MONTOAUTORIZADO'
]

tmp_df = df[df.AUTORIZACION.isin(autorizaciones)][tmp_df_columns]
tmp_df2 = df2[df2.AUTORIZACION.isin(autorizaciones)][tmp_df2_columns]

# agrupar montos en el dataset de distribución
tmp_df2_new = tmp_df2.groupby(['AUTORIZACION', 'CLIENTEAFILIADOBENEFICIARIO', 'PRESTACION', 'PRESTADOR', 
                               'FECHASERVICIO'])[['MONTOASEGURADORA', 'MONTOEXCEPCION', 'MONTOCONCESION', 'MONTOAUTORIZADO']].sum().reset_index()

# Consolidar las dos bases de datos
consolidated_df = df.merge(
    tmp_df2_new,
    on=['AUTORIZACION', 'CLIENTEAFILIADOBENEFICIARIO','PRESTACION', 'PRESTADOR', 'FECHASERVICIO'],
)

consolidated_df.drop(columns=['MONTOASEGURADORA_x'], inplace=True)
consolidated_df.rename(
    columns={
        'MONTOASEGURADORA_y': 'MONTOASEGURADORA',
        'MONTODIFERENCIAAFILIADO': 'MONTOUSUARIO',
        'TipoPrestador': 'TIPOPRESTADOR',
    },
    inplace=True
)


### Borrar variables pesadas

In [ ]:
del(df, df2, tmp_df, tmp_df2, tmp_df2_new)

---

# Preprocesamiento

In [ ]:
# Dar formato a Fechas de Servicio
consolidated_df['FECHASERVICIO'] = pd.to_datetime(consolidated_df['FECHASERVICIO'])

# Obtener mes de fecha de servicio
consolidated_df['FECHASERVICIO_MONTH'] = consolidated_df['FECHASERVICIO'].dt.month_name()
consolidated_df['FECHASERVICIO_MONTH_NUMBER'] = consolidated_df['FECHASERVICIO'].dt.month

# Eliminar espacios al final y al comienzo de Prestacion
consolidated_df['PRESTACION'] = consolidated_df['PRESTACION'].str.strip()

# Calcular Diferencia de montos reclamado y autorizado
consolidated_df['DIFERENCIA_MONTO_RECLAMADO_AUTORIZADO'] = consolidated_df['MONTORECLAMADO'] - consolidated_df['MONTOAUTORIZADO']

# Calcular Diferencia de montos reclamado y autorizado y usuario
consolidated_df['DIFERENCIA_MONTO_RECLAMADO_AUTORIZADO_USUARIO'] = consolidated_df['MONTORECLAMADO'] - (consolidated_df['MONTOAUTORIZADO'] + consolidated_df['MONTOUSUARIO'])

# Calcular Valores Absolutos
consolidated_df['MONTOAJUSTADO_ABS'] = abs(consolidated_df['MONTOAJUSTADO'])
consolidated_df['MONTOCONCESION_ABS'] = abs(consolidated_df['MONTOCONCESION'])
consolidated_df['DIFERENCIA_ABSOLUTA_MONTO_RECLAMADO_AUTORIZADO'] = abs(consolidated_df['DIFERENCIA_MONTO_RECLAMADO_AUTORIZADO'])

# Calcular Dias de habitación
dias_habitacion_mask = consolidated_df['PRESTACION'].isin(salas_list)
consolidated_df.loc[dias_habitacion_mask,'DIAS_HABITACION'] = consolidated_df['CANTIDAD']

# Calcular la fecha de fin de servicio
def calculate_end_date(ser):
    if ser['DIAS_HABITACION'] == np.nan:
        return

    start_date = ser['FECHASERVICIO']
    dias_habitacion = ser['CANTIDAD']
    end_date = start_date + datetime.timedelta(days=dias_habitacion)

    return end_date


consolidated_df['FECHASERVICIO_FIN'] = consolidated_df.apply(lambda x: calculate_end_date(x), axis=1)

In [ ]:
# Alertas para Reglas de Negocio, relacionada a montos

especialidades_claves = [
    'CENTROS MEDICOS',
    'ATENCION FARMACEUTICA', 
    'CTROS. DIAGNOSTICOS',
    'MEDICINA FISICA Y REHABILITACION', 
    'LABORATORIO CLINICO DE HEMATOLOGIA Y BAN',
]

consolidated_df['TIPO_ESPECIALIDAD'] = consolidated_df['ESPECIALIDAD']
consolidated_df.loc[~consolidated_df['TIPO_ESPECIALIDAD'].isin(especialidades_claves), 'TIPO_ESPECIALIDAD'] = 'OTROS'


# Acá voy 19/04/2023

In [ ]:
# Alertas para Reglas de Negocio, relacionada a montos
consolidated_df['ERROR_MONTO_RECLAMADO'] = (consolidated_df.DIFERENCIA_MONTO_RECLAMADO_AUTORIZADO_USUARIO != 0)* 1
consolidated_df['TIENE_MONTO_EXCEPCION'] = (abs(consolidated_df.MONTOEXCEPCION) > 0)*1
consolidated_df['ERROR_MONTO_AUTORIZADO2']=(abs(consolidated_df.MONTOAUTORIZADO - consolidated_df.MONTOASEGURADORA - consolidated_df.MONTOCONCESION - consolidated_df.MONTOEXCEPCION)>0.1)*1
consolidated_df['ERROR_CONSECION']=(consolidated_df.MONTOCONCESION_ABS > 0.2*consolidated_df.MONTOAUTORIZADO)*1
consolidated_df['ERROR_MONTO_AUTORIZADO1']=(consolidated_df.MONTOAUTORIZADO > consolidated_df.MONTORECLAMADO)*1

# Alertas para Reglas de Negocio, relacionada a los dias de habitación
consolidated_df['LIM_2_DIAS_HABITACION_EXCEDIDO']=(consolidated_df.DIAS_HABITACION>2)*1
consolidated_df['LIM_6_DIAS_HABITACION_EXCEDIDO']=(consolidated_df.DIAS_HABITACION>6)*1

# Indicadores de atención
consolidated_df['ATENCION_AMBULATORIA']=(consolidated_df.TIPOATENCION=='AMBULATORIA')*1
consolidated_df['ATENCION_HOSPITALARIA']=(consolidated_df.TIPOATENCION=='HOSPITALARIA')*1

# Indicadores de prestador
consolidated_df['CENTROS_MEDICOS']=(consolidated_df.TIPO_ESPECIALIDAD=='CENTROS MEDICOS')*1
consolidated_df['CENTRO_DIAGNOSTICO']=(consolidated_df.TIPO_ESPECIALIDAD=='CTROS. DIAGNOSTICOS')*1
consolidated_df['LABORATORIO_CLINICO']=(consolidated_df.TIPO_ESPECIALIDAD=='LABORATORIO CLINICO DE HEMATOLOGIA Y BAN')*1
consolidated_df['ATENCION_FARMACEUTICA']=(consolidated_df.TIPO_ESPECIALIDAD=='ATENCION FARMACEUTICA')*1
consolidated_df['OTRA_ESPECIALIDAD']=(consolidated_df.TIPO_ESPECIALIDAD=='OTROS')*1

consolidated_df['MEDICO_ESPECIALISTA']=(consolidated_df.TIPOPRESTADOR=='MEDICOS Y/O ESPECIALISTAS')*1
consolidated_df['ES_CONSULTA']=(consolidated_df['PRESTACION'].str.contains('CONSULTA'))*1

# Indicadores de procedimiento
consolidated_df['OTROS_SERVICIOS']=(consolidated_df.PRESTACION=='OTROS SERVICIOS NO CUBIERTOS')*1
consolidated_df['MEDICINAS_INTERNAMIENTO']=(consolidated_df.PRESTACION=='Medicinas Durante Internamiento todo tipo')*1
consolidated_df['MATERIALES_DESECHABLES']=(consolidated_df.PRESTACION=='MATERIALES DESECHABLES')*1

# Métricas
consolidated_df['MONTO_USUARIO_PER_CAPITA']=consolidated_df.MONTOUSUARIO/consolidated_df.CANTIDAD
consolidated_df['DIFERENCIA_RECLAMADO_AUTORIZADO']=consolidated_df.MONTORECLAMADO-consolidated_df.MONTOAUTORIZADO


In [ ]:
# Calcular prestaciones que exceden el maximo esperado.

consolidated_df = consolidated_df.set_index('PRESTACION').join(
    valores_referencia_cups_df[['cupsNombre','cupsMaxVecesDia']].set_index('cupsNombre'),
    how='left',
).reset_index().rename(columns={'index': 'PRESTACION'})


consolidated_df['LIM_CANTIDAD_PRESTACION_EXCEDIDO'] = (consolidated_df['CANTIDAD'] > consolidated_df['cupsMaxVecesDia'])*1

consolidated_df.drop(
    columns='cupsMaxVecesDia',
    inplace=True,
)

## Visita a diferente especialista, misma especialidad, mayor a 2 por mes 


In [ ]:
# Visita a diferente especialista, misma especialidad, mayor a 2 por mes 

cant_especialistas_diferentes = consolidated_df[
    (consolidated_df['MEDICO_ESPECIALISTA'] == 1) & (consolidated_df['ES_CONSULTA'] == 1)
].groupby(
    ['CLIENTEAFILIADOBENEFICIARIO', 'ESPECIALIDAD', 'FECHASERVICIO_MONTH_NUMBER', 'FECHASERVICIO_MONTH']
).agg(
    CANT_ESPECIALISTAS_DIFERENTES=pd.NamedAgg(column='PRESTADOR', aggfunc='nunique'),
    AUTORIZACIONES=pd.NamedAgg(column='AUTORIZACION', aggfunc='unique'),
).sort_values(by='CANT_ESPECIALISTAS_DIFERENTES', ascending=False)


cant_especialistas_diferentes = cant_especialistas_diferentes[cant_especialistas_diferentes['CANT_ESPECIALISTAS_DIFERENTES'] > 2]
cant_especialistas_diferentes.reset_index(inplace=True)

cant_especialistas_diferentes = cant_especialistas_diferentes.explode('AUTORIZACIONES')
autorizaciones_especialistas_diferentes = cant_especialistas_diferentes['AUTORIZACIONES'].unique()

consolidated_df['MISMA_ESPECIALIDAD_DIFF_ESPECIALISTA'] = (
    (consolidated_df['MEDICO_ESPECIALISTA'] == 1) & 
    (consolidated_df['ES_CONSULTA'] == 1) &
    (consolidated_df['AUTORIZACION'].isin(autorizaciones_especialistas_diferentes) )
)*1

In [ ]:
del(cant_especialistas_diferentes)

## Visita al mismo especialista mayor a 2 por mes


In [ ]:
# Visita al mismo especialista mayor a 2 por mes

cant_visitas_especialista = consolidated_df[
    (consolidated_df['MEDICO_ESPECIALISTA'] == 1) & 
    (consolidated_df['ES_CONSULTA'] == 1)
].groupby(
    ['CLIENTEAFILIADOBENEFICIARIO', 'PRESTADOR', 'FECHASERVICIO_MONTH_NUMBER', 'FECHASERVICIO_MONTH']
).agg(
    NUM_VISITAS=pd.NamedAgg(column='FECHASERVICIO', aggfunc='nunique'),
    AUTORIZACIONES=pd.NamedAgg(column='AUTORIZACION', aggfunc='unique'),
).sort_values(by='NUM_VISITAS', ascending=False)


cant_visitas_especialista = cant_visitas_especialista[cant_visitas_especialista['NUM_VISITAS'] > 2]
cant_visitas_especialista.reset_index(inplace=True)

cant_visitas_especialista = cant_visitas_especialista.explode('AUTORIZACIONES')
autorizaciones_visitas_especialista = cant_visitas_especialista['AUTORIZACIONES'].unique()

consolidated_df['LIM_CONSULTAS_EXCEDIDO_MES'] = (
    (consolidated_df['MEDICO_ESPECIALISTA'] == 1) & 
    (consolidated_df['ES_CONSULTA'] == 1) &
    (consolidated_df['AUTORIZACION'].isin(autorizaciones_visitas_especialista) )
)*1

In [ ]:
del(cant_visitas_especialista)

## Límite Emergencias excedido en el mes

In [ ]:
consolidated_df['ES_EMERGENCIA'] = (consolidated_df['SERVICIOSALUD'].str.contains('EMERGENCIA'))*1

freq_emergencias_mes = consolidated_df[
    consolidated_df['ES_EMERGENCIA'] == 1
].groupby(['CLIENTEAFILIADOBENEFICIARIO', 'FECHASERVICIO_MONTH_NUMBER', 'FECHASERVICIO_MONTH']).agg(
    NUM_AUTORIZACIONES_EMERGENCIAS_MES=pd.NamedAgg(column='AUTORIZACION', aggfunc='nunique'),
    AUTORIZACIONES=pd.NamedAgg(column='AUTORIZACION', aggfunc='unique'),
).sort_values(by='NUM_AUTORIZACIONES_EMERGENCIAS_MES', ascending=False)

freq_emergencias_mes = freq_emergencias_mes.explode('AUTORIZACIONES')
freq_emergencias_mes = freq_emergencias_mes[
    freq_emergencias_mes['NUM_AUTORIZACIONES_EMERGENCIAS_MES'] > 2
]

autorizaciones_emergencias_excedida = freq_emergencias_mes['AUTORIZACIONES'].unique()

consolidated_df['LIM_EMERGENCIAS_EXCEDIDO_MES'] = (
    (consolidated_df['ES_EMERGENCIA'] == 1) &
    (consolidated_df['AUTORIZACION'].isin(autorizaciones_emergencias_excedida) )
)*1

In [ ]:
del(freq_emergencias_mes)

## Consultas que no generan medicamentos

In [ ]:
autorizaciones_cond_consulta = consolidated_df[
    (consolidated_df['MEDICO_ESPECIALISTA'] == 1) & 
    (consolidated_df['ES_CONSULTA'] == 1)
]['AUTORIZACION'].values.tolist()
autorizaciones_cond_consulta = set(autorizaciones_cond_consulta)

In [ ]:
consultas_atipicas_df = consolidated_df[ consolidated_df['AUTORIZACION'].isin(autorizaciones_cond_consulta) ]
consultas_atipicas_df[['CLIENTEAFILIADOBENEFICIARIO', 'FECHASERVICIO']]
consultas_sample = consultas_atipicas_df[['CLIENTEAFILIADOBENEFICIARIO', 'FECHASERVICIO']].dropna()
consultas_sample['FECHASERVICIO'] = consultas_sample['FECHASERVICIO'] - pd.offsets.Day(16)
records = []

for _, row in consultas_sample.iterrows():
    tmp_usuario = row['CLIENTEAFILIADOBENEFICIARIO']
    tmp_fecha_inicio = row['FECHASERVICIO']
    
    tmp_df = pd.DataFrame()
    tmp_df['FECHASERVICIO'] = pd.date_range(tmp_fecha_inicio, periods=32, freq='D')
    tmp_df['CLIENTEAFILIADOBENEFICIARIO'] = tmp_usuario

    records += tmp_df.to_records(index=False).tolist()

new_consultas_dates_df = pd.DataFrame(
    records, 
    columns=['FECHASERVICIO', 'CLIENTEAFILIADOBENEFICIARIO'],
)

new_consultas_dates_df['FECHASERVICIO'] = pd.to_datetime(
    new_consultas_dates_df['FECHASERVICIO'], 
    errors='coerce'
)


In [ ]:
consultas_tracking_df = consolidated_df.merge(
    new_consultas_dates_df,
    on=['CLIENTEAFILIADOBENEFICIARIO', 'FECHASERVICIO']
)


sample = consultas_tracking_df.groupby(['CLIENTEAFILIADOBENEFICIARIO']).agg(
    NUM_PRESTACIONES=pd.NamedAgg(column='PRESTACION', aggfunc='count'),
)

usuarios = sample[sample['NUM_PRESTACIONES'] == 1].index.tolist()

sample_consultas_tracking_df = consultas_tracking_df[
    consultas_tracking_df['CLIENTEAFILIADOBENEFICIARIO'].isin(usuarios) 
]

sample_consultas_tracking_df = sample_consultas_tracking_df[
   sample_consultas_tracking_df['ESPECIALIDAD'] != 'PSICOLOGIA'
]

autorizaciones_cond_consulta_sin_lab_meds = sample_consultas_tracking_df['AUTORIZACION'].unique()

consolidated_df['CONSULTAS_SIN_LAB_MED'] = (
    consolidated_df['AUTORIZACION'].isin(autorizaciones_cond_consulta_sin_lab_meds)
)*1


In [ ]:
del(consultas_atipicas_df, new_consultas_dates_df, consultas_sample, consultas_tracking_df, sample_consultas_tracking_df)

## Distribución Medicamentos > 20%

In [ ]:
autorizaciones_padres_list = consolidated_df['AUTORIZACIONRELACIONADA'].dropna().unique().tolist()
consolidated_df.loc[ consolidated_df['AUTORIZACION'].isin(autorizaciones_padres_list) , 'AUTORIZACIONRELACIONADA' ] =  consolidated_df['AUTORIZACION']

eventos_agrupados_df = consolidated_df.groupby(
    ['AUTORIZACIONRELACIONADA', 'TIPOPRESTACION']
).agg(
    NUM_AUTORIZACIONES=pd.NamedAgg(column='AUTORIZACION', aggfunc='nunique'),
    NUM_PROCEDIMIENTOS=pd.NamedAgg(column='PRESTACION', aggfunc='count'),
    MONTO_RECLAMADO=pd.NamedAgg(column='MONTORECLAMADO', aggfunc='sum'),
    MONTO_AUTORIZADO=pd.NamedAgg(column='MONTOAUTORIZADO', aggfunc='sum'),
).sort_values(by=['MONTO_RECLAMADO'], ascending=False)


eventos_agrupados_pivot_df = eventos_agrupados_df.pivot_table(
    values=['MONTO_RECLAMADO'],
    columns=['TIPOPRESTACION'],
    index='AUTORIZACIONRELACIONADA'
).fillna(0)

eventos_agrupados_pivot_df['TOTAL'] = eventos_agrupados_pivot_df.sum(axis='columns')
eventos_agrupados_pivot_percent_df = (eventos_agrupados_pivot_df.T / eventos_agrupados_pivot_df['TOTAL'].T).T
eventos_agrupados_pivot_percent_df.columns = eventos_agrupados_pivot_percent_df.columns.droplevel(0)
eventos_agrupados_pivot_percent_df.reset_index(inplace=True)

In [ ]:

value_vars = eventos_agrupados_pivot_percent_df.columns.tolist()
id_vars = value_vars.pop(0)
eventos_melt =pd.melt(
    eventos_agrupados_pivot_percent_df,
    id_vars=id_vars,
    value_vars=value_vars,
)

eventos_melt = eventos_melt.rename(columns={'value': '% Total Monto del Evento'})
eventos_melt = eventos_melt[eventos_melt['TIPOPRESTACION'] != '']
eventos_melt = eventos_melt[eventos_melt['% Total Monto del Evento'] > 0]
eventos_melt_mask = (eventos_melt["% Total Monto del Evento"] > 0.2) & (eventos_melt["TIPOPRESTACION"].str.contains('MEDICIMENTOS') )
eventos_melt_outliers = eventos_melt[eventos_melt_mask]

autorizaciones_padres_dist_meds = eventos_melt_outliers.AUTORIZACIONRELACIONADA.unique()

consolidated_df['LIM_DIST_MEDS_EXCEDIDO'] = (
    consolidated_df['AUTORIZACIONRELACIONADA'].isin(autorizaciones_padres_dist_meds)
)*1


In [ ]:
del(eventos_melt, eventos_agrupados_pivot_percent_df, eventos_agrupados_pivot_df, eventos_agrupados_df)

---

# Preprocesamiento, Fase II

In [ ]:
import scipy.stats as stats

Montos=[
    'MONTOAJUSTADO', 
    'MONTOUSUARIO',
    'MONTOASEGURADORA',
    'MONTOEXCEPCION',
    'MONTOCONCESION',
    'MONTOAUTORIZADO',
    'MONTOAJUSTADO_ABS',
    'MONTOCONCESION_ABS',
    'MONTO_USUARIO_PER_CAPITA'
]


for i in Montos:
    x_df = consolidated_df[i]
    Q1 = x_df.quantile(q=.25)
    Q3 = x_df.quantile(q=.75)
    IQR = Q3-Q1
    consolidated_df['ATIPICO_'+i]=((x_df < (Q1-2*IQR)) | (x_df > (Q3+2*IQR)))*1

consolidated_df.ERROR_CONSECION=(consolidated_df.ERROR_CONSECION-1)*(-1)

---

## Escribir Archivo

In [ ]:
consolidated_df.to_parquet('../data/processed/ConsolidatedDataAutorizacionesNov2021Mar2022_features.parquet')

---

# Cálculo de las métricas previas a la predicción

In [ ]:
measures = {
    'CANTIDAD_AUTORIZACIONES': pd.NamedAgg(column='AUTORIZACION', aggfunc='nunique'),
    'LIM_CANTIDAD_PRESTACION_EXCEDIDO': pd.NamedAgg(column='LIM_CANTIDAD_PRESTACION_EXCEDIDO', aggfunc='sum'), #
    'MISMA_ESPECIALIDAD_DIFF_ESPECIALISTA': pd.NamedAgg(column='MISMA_ESPECIALIDAD_DIFF_ESPECIALISTA', aggfunc='sum'), #
    'LIM_CONSULTAS_EXCEDIDO_MES': pd.NamedAgg(column='LIM_CONSULTAS_EXCEDIDO_MES', aggfunc='sum'), #
    'ES_EMERGENCIA': pd.NamedAgg(column='ES_EMERGENCIA', aggfunc='sum'),  
    'LIM_EMERGENCIAS_EXCEDIDO_MES': pd.NamedAgg(column='LIM_EMERGENCIAS_EXCEDIDO_MES', aggfunc='sum'), #     
    'ATIPICO_MONTO_ASEGURADORA': pd.NamedAgg(column='ATIPICO_MONTOASEGURADORA', aggfunc='sum'), # N/A
    'ATIPICO_MONTO_CONCESION': pd.NamedAgg(column='ATIPICO_MONTOCONCESION', aggfunc='sum'), # N/A
    'ATIPICO_MONTO_EXCEPCION': pd.NamedAgg(column='ATIPICO_MONTOEXCEPCION', aggfunc='sum'), # N/A
    'ATIPICO_MONTO_USUARIO': pd.NamedAgg(column='ATIPICO_MONTOUSUARIO', aggfunc='sum'), # N/A
    'ATIPICO_MONTO_AJUSTADO': pd.NamedAgg(column='ATIPICO_MONTOAJUSTADO', aggfunc='sum'), # N/A
    'ATIPICO_MONTO_USUARIO_PER_CAPITA': pd.NamedAgg(column='ATIPICO_MONTO_USUARIO_PER_CAPITA', aggfunc='sum'), #N/A
    'ERRORES_MONTO_RECLAMADO': pd.NamedAgg(column='ERROR_MONTO_RECLAMADO', aggfunc='sum'),
    'ERRORES_MONTO_AUTORIZADO1': pd.NamedAgg(column='ERROR_MONTO_AUTORIZADO1', aggfunc='sum'),
    'ERRORES_MONTO_AUTORIZADO2': pd.NamedAgg(column='ERROR_MONTO_AUTORIZADO2', aggfunc='sum'),
    'CANTIDAD_DIAS_HABITACION': pd.NamedAgg(column='DIAS_HABITACION', aggfunc='sum'),
    'LIM_6_DIAS_HABITACION_EXCEDIDO': pd.NamedAgg(column='LIM_6_DIAS_HABITACION_EXCEDIDO', aggfunc='sum'),
    'PRESTACION_MEDICINAS_INTERNAMIENTO': pd.NamedAgg(column='MEDICINAS_INTERNAMIENTO', aggfunc='sum'),
    'PRESTACION_MATERIALES_DESECHABLES': pd.NamedAgg(column='MATERIALES_DESECHABLES', aggfunc='sum'),
    'PRESTACION_OTROS_SERVICIOS': pd.NamedAgg(column='OTROS_SERVICIOS', aggfunc='sum'),
    'N_PRESTACIONES': pd.NamedAgg(column='PRESTACION', aggfunc='count'),        
    'ERRORES_CONSECION': pd.NamedAgg(column='ERROR_CONSECION', aggfunc='sum'),
    'CONSULTAS_SIN_LAB_MED': pd.NamedAgg(column='CONSULTAS_SIN_LAB_MED', aggfunc='sum'), #
    'LIM_DIST_MEDS_EXCEDIDO': pd.NamedAgg(column='LIM_DIST_MEDS_EXCEDIDO', aggfunc='sum') #
   }

df_medidas = consolidated_df.groupby(
    ['PRESTADOR', 'TipoInstitucionPrestador']
).agg(
    **measures
).sort_values(
    by='CANTIDAD_AUTORIZACIONES', 
    ascending=False
).reset_index()

## Escribir Archivo

In [ ]:
df_medidas.to_parquet('../data/processed/summaryPrestadores_to_predict.parquet')

# Predecir

## Cargar Modelo

In [ ]:
# load the model from disk
#model_filename = '../models/et_model_prestadores_atipicos.pkl'
#model = pickle.load(open(model_filename, 'rb'))

with open('../models/et_model_prestadores_atipicos.pkl', 'rb') as f:
    model = pickle.load(f)

/anaconda/envs/azureml_py38/lib/python3.8/site-packages/sklearn/base.py:313: UserWarning: Trying to unpickle estimator ExtraTreeClassifier from version 1.1.3 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/anaconda/envs/azureml_py38/lib/python3.8/site-packages/sklearn/base.py:313: UserWarning: Trying to unpickle estimator ExtraTreesClassifier from version 1.1.3 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [ ]:
df_medidas.set_index(['PRESTADOR', 'TipoInstitucionPrestador'], inplace=True)

scaler = StandardScaler()
scaler.fit(df_medidas)

X_normalized = scaler.transform(df_medidas.copy())
X_normalized = pd.DataFrame(X_normalized, index=df_medidas.index, columns=df_medidas.columns)
X_normalized = pd.get_dummies(X_normalized.reset_index('TipoInstitucionPrestador'))

# Solo en el caso que aparezca
X_normalized.drop(columns='TipoInstitucionPrestador_NO INFORMA', inplace=True)

In [ ]:
X_normalized.describe()

,CANTIDAD_AUTORIZACIONES,LIM_CANTIDAD_PRESTACION_EXCEDIDO,MISMA_ESPECIALIDAD_DIFF_ESPECIALISTA,LIM_CONSULTAS_EXCEDIDO_MES,ES_EMERGENCIA,LIM_EMERGENCIAS_EXCEDIDO_MES,ATIPICO_MONTO_ASEGURADORA,ATIPICO_MONTO_CONCESION,ATIPICO_MONTO_EXCEPCION,ATIPICO_MONTO_USUARIO,...,ERRORES_CONSECION,CONSULTAS_SIN_LAB_MED,LIM_DIST_MEDS_EXCEDIDO,TipoInstitucionPrestador_CAPITACION,TipoInstitucionPrestador_CENTROS DIAGNÓSTICOS,TipoInstitucionPrestador_CENTROS MÉDICOS,TipoInstitucionPrestador_FARMACIAS,TipoInstitucionPrestador_HOSPITALARIA,TipoInstitucionPrestador_LABORATORIO,TipoInstitucionPrestador_MÉDICOS
count,5.825000e+03,5.825000e+03,5.825000e+03,5.825000e+03,5.825000e+03,5.825000e+03,5.825000e+03,5.825000e+03,5825.0,5.825000e+03,...,5.825000e+03,5.825000e+03,5.825000e+03,5825.000000,5825.000000,5825.000000,5825.000000,5825.000000,5825.000000,5825.000000
mean,-2.896777e-16,-4.205835e-15,-8.636773e-16,-1.996539e-14,6.308864e-15,4.068963e-15,-2.266051e-15,-4.467733e-15,0.0,1.170518e-15,...,-3.116487e-17,-5.025679e-15,-3.920807e-15,0.003090,0.030558,0.033305,0.087554,0.002918,0.010472,0.827468
std,1.000086e+00,1.000086e+00,1.000086e+00,1.000086e+00,1.000086e+00,1.000086e+00,1.000086e+00,1.000086e+00,0.0,1.000086e+00,...,1.000086e+00,1.000086e+00,1.000086e+00,0.055508,0.172131,0.179446,0.282669,0.053948,0.101805,0.377875
min,-1.750671e-01,-1.061079e-01,-1.392070e-01,-3.374295e-01,-7.353591e-02,-7.212152e-02,-1.706573e-01,-9.910568e-02,0.0,-1.879337e-01,...,-6.217611e-02,-4.837008e-01,-7.583333e-02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,-1.615493e-01,-1.061079e-01,-1.392070e-01,-3.374295e-01,-7.353591e-02,-7.212152e-02,-1.680713e-01,-9.910568e-02,0.0,-1.879337e-01,...,-5.992615e-02,-4.837008e-01,-7.583333e-02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,-1.353586e-01,-1.061079e-01,-1.392070e-01,-3.374295e-01,-7.353591e-02,-7.212152e-02,-1.551411e-01,-9.910568e-02,0.0,-1.782753e-01,...,-5.598873e-02,-2.594087e-01,-7.583333e-02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,-7.537333e-02,-1.061079e-01,-1.392070e-01,-3.374295e-01,-7.353591e-02,-7.212152e-02,-1.111784e-01,-5.460629e-02,0.0,-1.428612e-01,...,-4.665142e-02,1.891754e-01,-6.122411e-02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,4.530218e+01,3.258516e+01,2.882083e+01,1.560617e+01,4.253604e+01,3.630430e+01,2.580611e+01,3.892686e+01,0.0,2.393871e+01,...,5.850729e+01,2.284267e+01,4.160426e+01,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


## Predecir Etiquetas

In [ ]:

display(model)

/anaconda/envs/azureml_py38/lib/python3.8/site-packages/sklearn/base.py:193: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  warnings.warn('From version 0.24, get_params will raise an '


ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=None, max_features='auto',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
                     oob_score=False, random_state=6045, verbose=0,
                     warm_start=False)

In [ ]:
y_predicted = model.predict(X_normalized)
df_medidas['Atipicidad'] = y_predicted

AttributeError: 'ExtraTreeClassifier' object has no attribute 'n_features_'

## Guardar Predicciones

In [ ]:
df_medidas.to_csv('../data/processed/summaryPrestadores_predicted.csv')